In [1]:
import requests
import logging
from typing import Any, Dict, List
from pyspark.sql import SparkSession
import json

In [2]:
minio_connection = ""

In [3]:
# move to functinon
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open('../variables/minio_connection.json', "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [4]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
        "com.amazonaws:aws-java-sdk-bundle:1.12.787",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            .master(master_url)
        )

        return builder.getOrCreate()

In [5]:
spark = LazySparkSession(
    access_key=minio_conn.get("access_key"), 
    secret_key=minio_conn.get("key"), 
    endpoint=minio_conn.get("endpoint")
).start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 16:02:38 WARN Utils: Your hostname, DESKTOP-EDEM2DH, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/24 16:02:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/edcarlos/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/edcarlos/.ivy2.5.2/cache
The jars for the packages stored in: /home/edcarlos/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1081ba58-c0b4-4ba5-8381-d30a6f9449ab;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central

In [6]:
# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [7]:
def fetch_ibge_ipca_amplo_data(url: str) -> List[Dict[str, Any]] | None:
    """
    Realiza a requisição HTTP e valida a resposta, com tratamento de erros e logs.
    """
    try:
        logging.info(f"Tentando requisição para: {url}")
        response = requests.get(url) 

    except requests.exceptions.ConnectionError:
        logging.error(f"Erro de conexão ao acessar {url}. Verifique sua rede.")
        return None
    except requests.exceptions.HTTPError as e:
        logging.error(f"Erro HTTP na requisição para {url}: {e}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Erro inesperado na requisição para {url}: {e}")
        return None
    
    content_type = response.headers.get('Content-Type', '')
    if 'application/json' not in content_type:
        logging.error(f" Resposta não é JSON: {content_type}")
        return None
        
    if len(response.content) == 0:
        logging.error(" Resposta vazia")
        return None
        
    logging.info("Requisição OK.")
    return response.json()

In [8]:
agregados = 7060
periodos = "202001|202002" 
variaveis = "63|69|2265|66"
localidades = "N1[all]|N6[all]"
classificacao = "315[all]"

In [9]:
url = (
    f"https://servicodados.ibge.gov.br/api/v3/agregados/{agregados}"
    f"/periodos/{periodos}"
    f"/variaveis/{variaveis}"
    f"?localidades={localidades}"
    f"&classificacao={classificacao}"
)

In [10]:

print("\n--- Requisição com 2 Períodos (202001|202002) ---")

response = fetch_ibge_ipca_amplo_data(url)

if response:
    print(f"Requisição OK. Total de registros recebidos: {len(response)}")

    # Visualização resumida: primeiros 2 registros formatados
    for i, item in enumerate(response[:2], start=1):
        print(f"\nRegistro {i}:")
        for k, v in item.items():
            print(f"  {k}: {v}")
else:
    print("A requisição falhou ou os dados foram invalidados.")

2025-07-24 16:02:56,149 - INFO - Tentando requisição para: https://servicodados.ibge.gov.br/api/v3/agregados/7060/periodos/202001|202002/variaveis/63|69|2265|66?localidades=N1[all]|N6[all]&classificacao=315[all]



--- Requisição com 2 Períodos (202001|202002) ---


2025-07-24 16:02:56,607 - INFO - Requisição OK.


Requisição OK. Total de registros recebidos: 4

Registro 1:
  id: 63
  variavel: IPCA - Variação mensal
  unidade: %
  resultados: [{'classificacoes': [{'id': '315', 'nome': 'Geral, grupo, subgrupo, item e subitem', 'categoria': {'7169': 'Índice geral'}}], 'series': [{'localidade': {'id': '1', 'nivel': {'id': 'N1', 'nome': 'Brasil'}, 'nome': 'Brasil'}, 'serie': {'202001': '0.21', '202002': '0.25'}}, {'localidade': {'id': '1200401', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Rio Branco - AC'}, 'serie': {'202001': '-0.21', '202002': '0.49'}}, {'localidade': {'id': '2111300', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'São Luís - MA'}, 'serie': {'202001': '-0.19', '202002': '0.18'}}, {'localidade': {'id': '2800308', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Aracaju - SE'}, 'serie': {'202001': '0.39', '202002': '0.66'}}, {'localidade': {'id': '5002704', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Campo Grande - MS'}, 'serie': {'202001': '0.13', '20200

In [11]:
response[0].keys()

dict_keys(['id', 'variavel', 'unidade', 'resultados'])

In [12]:
"""
Navega por um JSON estruturado em: resultados → classificacoes + series

Faz joins manuais entre:

classificações ↔ categorias

localidades ↔ níveis

séries ↔ períodos e valores
"""

def safe_float(valor):
    try:
        return float(valor)
    except (ValueError, TypeError):
        return None

registros_extraidos = []

for resposta in response:
    for resultado in resposta['resultados']:
        classificacoes = resultado['classificacoes']
        series_temporais = resultado['series']

# Para cada classificação, iteramos todas as séries temporais
# para gerar todas as combinações possíveis (produto cartesiano)

        for classificacao in classificacoes:
            classificacao_copia = classificacao.copy()
            categorias = classificacao_copia.pop('categoria')

            for categoria_id, categoria_nome in categorias.items():
                classificacao_id = classificacao_copia['id']
                classificacao_nome = classificacao_copia['nome']

                for serie_temporal in series_temporais:
                    localidade = serie_temporal['localidade']
                    serie = serie_temporal['serie']

                    localidade_copia = localidade.copy()
                    nivel_info = localidade_copia.pop('nivel')

                    localidade_detalhada = {
                        **localidade_copia,
                        'nivel_id': nivel_info['id'],
                        'nivel_nome': nivel_info['nome']
                    }

                    for periodo, valor in serie.items():
                        registro = {
                            "id_variavel": resposta["id"],
                            "nome_variavel": resposta["variavel"],
                            "unidade_medida": resposta["unidade"],
                            "id_classificacao": classificacao_id,
                            "nome_classificacao": classificacao_nome,
                            "id_categoria": categoria_id,
                            "nome_categoria": categoria_nome,
                            "id_localidade": localidade_detalhada['id'],
                            "nome_localidade": localidade_detalhada['nome'],
                            "id_nivel": localidade_detalhada['nivel_id'],
                            "nome_nivel": localidade_detalhada['nivel_nome'],
                            "periodo": periodo,
                            "valor": safe_float(valor)
                        }
                        registros_extraidos.append(registro)



In [13]:
df = spark.createDataFrame(registros_extraidos)

In [14]:
df.write.format("delta").mode("overwrite").save("s3a://landing/ibge/ipca_amplo")

25/07/24 16:02:59 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
25/07/24 16:03:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/24 16:03:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/07/24 16:03:09 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to ibge/ipca_amplo/part-00006-5d07c927-e48e-4a41-8168-87ac37110563-c000.snappy.parquet. This is Unsupported


In [15]:
spark.read.format("delta").load("s3a://landing/ibge/ipca_amplo").show()

+------------+----------------+-------------+--------+-----------+--------------------+--------------------+-----------------+----------+--------------------+-------+--------------+-----+
|id_categoria|id_classificacao|id_localidade|id_nivel|id_variavel|      nome_categoria|  nome_classificacao|  nome_localidade|nome_nivel|       nome_variavel|periodo|unidade_medida|valor|
+------------+----------------+-------------+--------+-----------+--------------------+--------------------+-----------------+----------+--------------------+-------+--------------+-----+
|       47636|             315|      2800308|      N6|         63|2104020.Limpador ...|Geral, grupo, sub...|     Aracaju - SE| Município|IPCA - Variação m...| 202001|             %| NULL|
|       47636|             315|      2800308|      N6|         63|2104020.Limpador ...|Geral, grupo, sub...|     Aracaju - SE| Município|IPCA - Variação m...| 202002|             %| NULL|
|       47636|             315|      5002704|      N6|      